<a href="https://colab.research.google.com/github/KenObata/TISMIR_notebooks/blob/main/week15_BERT_base_keras_KFold_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Week15: This notebook uses Pre-Trained BERT

Situation: English only (=multi-class).
Split: StratifiedKfold.
Reference: https://github.com/jasonwei20/eda_nlp


### set up

In [1]:
!pip3 install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 6.4 MB/s 


In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from collections import Counter

from skmultilearn.model_selection import IterativeStratification   
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix

In [3]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score
DIR = '/content/drive/MyDrive/music4all/'
def get_balanced_accuracy(model, McNemar, is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning):
  test_y = test.map(map_func_only_y)
  y_category=np.zeros((TEST_SIZE, ))
  counter=0
  for label_tensor in test_y.take(len(test_y)):
    y_test = np.argmax(label_tensor, axis=1)
    for label in y_test:
      y_category[counter]=label
      counter+=1

  X_test, y_test = test.map(map_func_only_X), y_category
  y_predict_test = np.asarray(model.predict(X_test))
  y_predict_test = np.argmax(y_predict_test, axis=1)
  print(classification_report(y_test, y_predict_test) )
  print(balanced_accuracy_score(y_test, y_predict_test))

  McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)] = []
  for ground_truh, pred in zip(y_test, y_predict_test):
        if ground_truh==pred:
          McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)].append(True)
        else:
          McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)].append(False)
  with open(DIR+ "BERT_simple_log.txt", "a") as f:
    print("======================================", file=f)
    print("is_fine_tuning?:", is_fine_tuning, "drop_out_rate: ", drop_out_rate, "learning_rate_transfer_learning: ", learning_rate_transfer_learning,
          "learning_rate_fine_tuning: ", learning_rate_fine_tuning, file=f)
    print(classification_report(y_test, y_predict_test) , file=f)
    print(balanced_accuracy_score(y_test, y_predict_test), file=f)

  return balanced_accuracy_score(y_test, y_predict_test), McNemar

In [4]:
#save output into text
DIR = '/content/drive/MyDrive/music4all/'

### Data Preparation(Kfold split)

Create dataframe for Gensim

In [5]:
import pandas as pd

DIR = '/content/drive/MyDrive/music4all/'
df_genre_by_lang = pd.read_csv(DIR + 'df_genre_by_lang_full.csv')
df_genre_by_lang

,Unnamed: 0,id,genres,lang,lyric,number_of_line
0,0,0009fFIM1eYThaPg,pop,en,a sunny day so I got nowhere to hide Not a clo...,91
1,1,00P2bHdWFkghmDqz,soul,en,Tell me a tale that always was Sing me a song ...,36
2,2,00b6fV3nx5z2b8Ls,pop,en,A buh A buh You went to school to learn girl T...,74
3,3,013QDoTqbexEwkHr,pop,en,like a conversation where stops to breathe Is ...,20
4,4,01EKNot8qVgZpKM7,rock,en,Say the words I cannot say Say them on another...,31
...,...,...,...,...,...,...
13535,13535,zzT504Z94j1IAuc3,indie rock,en,think what afraid of come in you know been mad...,18
13536,13536,zzgS4ZqyswamEWNj,pop,en,Oh yeah yeah Last night I took a walk in the s...,75
13537,13537,zzx8CWdM7qkxKQpC,indie rock,en,Innocence it come easy in a sense it never wil...,34
13538,13538,zzz0n04uuTUA7fNh,pop,en,Girl you know how I feel I really Since you be...,65


In [6]:
def load_data(df_col, y):
    texts, labels = [], []
    
    for line in df_col:
        # texts are already tokenized, just split on space
        # in a real use-case we would put more effort in preprocessing
        texts.append(line.split(' '))
    return pd.DataFrame({'texts': texts, 'labels': y})

In [7]:
data = load_data(df_genre_by_lang["lyric"], df_genre_by_lang["genres"])

In [8]:
data

,texts,labels
0,"[a, sunny, day, so, I, got, nowhere, to, hide,...",pop
1,"[Tell, me, a, tale, that, always, was, Sing, m...",soul
2,"[A, buh, A, buh, You, went, to, school, to, le...",pop
3,"[like, a, conversation, where, stops, to, brea...",pop
4,"[Say, the, words, I, cannot, say, Say, them, o...",rock
...,...,...
13535,"[think, what, afraid, of, come, in, you, know,...",indie rock
13536,"[Oh, yeah, yeah, Last, night, I, took, a, walk...",pop
13537,"[Innocence, it, come, easy, in, a, sense, it, ...",indie rock
13538,"[Girl, you, know, how, I, feel, I, really, Sin...",pop


In [9]:
data['labels'] = data['labels'].astype('category')
label_mapping = data['labels'].cat.categories
data['labels'] = data['labels'].cat.codes
X = data['texts']
y = data['labels']

In [10]:
type(X)

pandas.core.series.Series

In [11]:
def StratifiedKFold_feature_and_df_glove(df, feature_list, y_name):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1209)  # 20% for test set 
  y = df[y_name]
  skf.get_n_splits(df[ feature_list ], y)

  splits = []

  for train_index, test_index in skf.split(df[ feature_list ], y):
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = df[ feature_list ].loc[train_index], df[ feature_list ].loc[test_index]
      y_train, y_test = y.loc[train_index], y.loc[test_index]
      splits.append({'X_train': X_train, 'X_test': X_test, 'y_train':y_train, 'y_test':y_test })
  return splits

In [12]:
def StratifiedKFold_feature_and_df(X, y):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1209)  # 20% for test set 
  #y = df[y_name]
  skf.get_n_splits(X, y)#df[ feature_list ]

  splits = []

  for train_index, test_index in skf.split(X, y):#df[ feature_list ]
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = X.loc[train_index], X.loc[test_index]
      y_train, y_test = y.loc[train_index], y.loc[test_index]
      splits.append({'X_train': X_train, 'X_test': X_test, 'y_train':y_train, 'y_test':y_test })
  return splits

In [13]:
#feature_list = ["texts"] #this is BOW and TF-IDF
#splits = StratifiedKFold_feature_and_df( data, feature_list, 'labels')
splits = StratifiedKFold_feature_and_df( X,y)

TRAIN: [    0     1     3 ... 13537 13538 13539] TEST: [    2     4     5 ... 13526 13532 13535]
TRAIN: [    0     2     4 ... 13535 13536 13539] TEST: [    1     3     7 ... 13530 13537 13538]
TRAIN: [    0     1     2 ... 13537 13538 13539] TEST: [    8    14    22 ... 13521 13531 13536]
TRAIN: [    0     1     2 ... 13537 13538 13539] TEST: [   10    12    15 ... 13523 13525 13534]
TRAIN: [    1     2     3 ... 13536 13537 13538] TEST: [    0     6    11 ... 13529 13533 13539]


In [14]:
len(splits)

5

In [15]:
split0=splits[0]
print(split0['X_train'].shape)
print(split0['y_train'].shape)
print(split0['X_test'].shape)
print(split0['y_test'].shape)

(10832,)
(10832,)
(2708,)
(2708,)


In [16]:
split0['X_train']

0        [a, sunny, day, so, I, got, nowhere, to, hide,...
1        [Tell, me, a, tale, that, always, was, Sing, m...
3        [like, a, conversation, where, stops, to, brea...
6        [Locked, up, tight, Like, I, would, never, fee...
7        [sittin, in, the, crib, dreamin, about, leer, ...
                               ...                        
13534    [grandma, cookies, nigga, Shout, out, to, fron...
13536    [Oh, yeah, yeah, Last, night, I, took, a, walk...
13537    [Innocence, it, come, easy, in, a, sense, it, ...
13538    [Girl, you, know, how, I, feel, I, really, Sin...
13539    [wwI, oh, must, go, on, standing, You, break, ...
Name: texts, Length: 10832, dtype: object

In [17]:
split0['y_train']

0        4
1        9
3        4
6        4
7        6
        ..
13534    6
13536    4
13537    3
13538    4
13539    8
Name: labels, Length: 10832, dtype: int8

### Use my self programmed balanced accuracy

In [ ]:
metrics = Metrics()
history = model.fit(train, validation_data=val, epochs=10, class_weight=my_weight ,callbacks=[metrics])
metrics.get_data()

Epoch 1/10
339/339 [==============================] - 225s 645ms/step - loss: 0.5565 - categorical_accuracy: 0.3878 - val_loss: 1.9337 - val_categorical_accuracy: 0.3273
Epoch 2/10
339/339 [==============================] - 218s 645ms/step - loss: 0.5397 - categorical_accuracy: 0.3626 - val_loss: 2.0651 - val_categorical_accuracy: 0.2764
Epoch 3/10
339/339 [==============================] - 218s 645ms/step - loss: 0.5375 - categorical_accuracy: 0.3392 - val_loss: 2.0931 - val_categorical_accuracy: 0.2273
Epoch 4/10
339/339 [==============================] - 218s 645ms/step - loss: 0.5378 - categorical_accuracy: 0.3197 - val_loss: 2.0576 - val_categorical_accuracy: 0.2459
Epoch 5/10
339/339 [==============================] - 219s 645ms/step - loss: 0.5333 - categorical_accuracy: 0.3077 - val_loss: 2.0292 - val_categorical_accuracy: 0.2816
Epoch 6/10
339/339 [==============================] - 219s 646ms/step - loss: 0.5373 - categorical_accuracy: 0.2967 - val_loss: 2.1738 - val_categoric

[{'balanced_accuracy': 0.09744849916749872},
 {'balanced_accuracy': 0.10420636932765996},
 {'balanced_accuracy': 0.10970405736872939},
 {'balanced_accuracy': 0.10356561320774078},
 {'balanced_accuracy': 0.10792864804613309},
 {'balanced_accuracy': 0.09168989857066105},
 {'balanced_accuracy': 0.09926135499834263},
 {'balanced_accuracy': 0.10014630125778516},
 {'balanced_accuracy': 0.09807505457274496},
 {'balanced_accuracy': 0.09881853716030842}]

## From here, separate X_train, X_test from KFOldSplit

### Preprocess my lyrics data (Official train and test)

In [18]:
import tensorflow as tf
!pip3 install transformers
SEQ_LEN = 256#512

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 14.6 MB/s 
     |████████████████████████████████| 120 kB 78.4 MB/s 
     |████████████████████████████████| 6.6 MB 65.9 MB/s 


In [19]:
split0['X_test']

2        [A, buh, A, buh, You, went, to, school, to, le...
4        [Say, the, words, I, cannot, say, Say, them, o...
5        [I, was, alone, I, was, made, of, stone, You, ...
9        [Again, the, burden, of, losing, rests, upon, ...
20       [only, been, three, weeks, And, a, bag, of, sp...
                               ...                        
13517    [Like, the, legend, of, the, Phoenix, All, end...
13522    [Mr, Telephone, man, something, wrong, with, m...
13526    [can, you, imagine, what, it, would, be, like,...
13532    [Love, of, my, life, hurt, me, broken, my, hea...
13535    [think, what, afraid, of, come, in, you, know,...
Name: texts, Length: 2708, dtype: object

In [20]:
def prepare_lyrics(X_series):
  for i, token_list in X_series.items():
    if type(token_list) is list:
      X_series.loc[i] = ' '.join(token_list)
  return X_series

In [21]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
SEQ_LEN=256

def get_Xid_Xmask(X_origin):
  Xids_train = np.zeros((X_origin.shape[0], SEQ_LEN))
  Xmask_train = np.zeros((X_origin.shape[0], SEQ_LEN))

  for i, lyric in enumerate(X_origin):
    tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
    Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']
  return Xids_train, Xmask_train

In [22]:
def map_func(input_ids, masks, labels):
  return {'input_ids': input_ids, 'attention_mask':masks}, labels

In [23]:
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
#tokenizer = AlbertTokenizer.from_pretrained('albert-large-v2')
Xids_train, Xmask_train = get_Xid_Xmask(split0['X_train'])
Xids_test, Xmask_test = get_Xid_Xmask(split0['X_test'])

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))
dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)

dataset_train = dataset_train.shuffle(42).batch(16)
DS_LEN = len(list(dataset_train))
SPLIT=0.9
train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

In [24]:
counter = Counter(split0['y_train'])
SUM=0
for item in list(counter.values()) :
  SUM+=item
#SUM = sum(counter.values())
SUM

10832

In [25]:
#Tutorial
#weight_for_0 = (1 / neg) * (total / 2.0)
#weight_for_1 = (1 / pos) * (total / 2.0)

counter = Counter(split0['y_train'])
my_weight2 = {}
print(counter)

for genre in counter:
  #print(genre, counter[genre])
  my_weight2[genre] = (1/counter[genre]) * (SUM/10)
my_weight2

Counter({4: 4143, 7: 1159, 9: 1030, 3: 865, 6: 783, 0: 763, 1: 690, 8: 556, 2: 537, 5: 306})


{4: 0.2614530533429882,
 9: 1.051650485436893,
 6: 1.383397190293742,
 2: 2.0171322160148977,
 0: 1.4196592398427261,
 7: 0.9345987920621226,
 1: 1.569855072463768,
 5: 3.539869281045752,
 8: 1.948201438848921,
 3: 1.2522543352601156}

In [26]:
TEST_SIZE = len(split0['X_test'])

In [27]:
def map_func_only_X(val_dictionary, labels):
  return {'input_ids': val_dictionary['input_ids'], 'attention_mask':val_dictionary['attention_mask']}

In [28]:
def map_func_only_y(val_dictionary, labels):
  return labels

### First Fold Do parameter tuning for dropout rate

In [ ]:
del(dataset_train)
del(dataset_test)

In [ ]:
from transformers import AlbertTokenizer, TFAlbertModel
import numpy as np

split0=splits[0]
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
SEQ_LEN=256
print(split0['X_train'].shape, split0['X_test'].shape)

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']

print("Xids_train.shape, Xids_test.shape: ",Xids_train.shape, Xids_test.shape)

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)
dataset_train = dataset_train.shuffle(42).batch(16)

DS_LEN = len(list(dataset_train))
SPLIT = 0.9
train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

(10832,) (2708,)
Xids_train.shape, Xids_test.shape:  (10832, 256) (2708, 256)


I have intentionally separated cells for check point purpose, based on dropout rates

In [ ]:
seen_parameter = []#(drop_out_rate, initial_rate, fine_tune_learning_rate)

In [ ]:
McNemar={}
balanced_accuracies_transfer_learning=[]
balanced_accuracies_fine_tuning = []

In [ ]:
del(model1)

NameError: ignored

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.1
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = bert_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    print(model1.summary())

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.1


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

609/609 [==============================] - 204s 309ms/step - loss: 1.7137 - categorical_accuracy: 0.3321 - val_loss: 2.0782 - val_categorical_accuracy: 0.2243
Epoch 2/6
609/609 [==============================] - 185s 304ms/step - loss: 1.5607 - categorical_accuracy: 0.3592 - val_loss: 2.0861 - val_categorical_accuracy: 0.2500
Epoch 3/6
609/609 [==============================] - 185s 303ms/step - loss: 1.4482 - categorical_accuracy: 0.4028 - val_loss: 2.1160 - val_categorical_accuracy: 0.2233
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 1.3561 - categorical_accuracy: 0.4269 - val_loss: 1.9283 - val_categorical_accuracy: 0.2767
Epoch 5/6
609/609 [==============================] - 185s 304ms/step - loss: 1.2561 - categorical_accuracy: 0.4597 - val_loss: 1.8924 - val_categorical_accuracy: 0.2923
Epoch 6/6
609/609 [==============================] - 185s 303ms/step - loss: 1.1755 - categorical_accuracy: 0.4896 - val_loss: 1.9968 - val_categorical_accuracy: 0.2

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


...

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model_2 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

609/609 [==============================] - 205s 308ms/step - loss: 1.9895 - categorical_accuracy: 0.2717 - val_loss: 2.2086 - val_categorical_accuracy: 0.2059
Epoch 2/6
609/609 [==============================] - 185s 304ms/step - loss: 1.8778 - categorical_accuracy: 0.2936 - val_loss: 2.1508 - val_categorical_accuracy: 0.2325
Epoch 3/6
609/609 [==============================] - 185s 304ms/step - loss: 1.7795 - categorical_accuracy: 0.3141 - val_loss: 2.0937 - val_categorical_accuracy: 0.2546
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 1.6954 - categorical_accuracy: 0.3346 - val_loss: 2.0743 - val_categorical_accuracy: 0.2555
Epoch 5/6
609/609 [==============================] - 185s 304ms/step - loss: 1.5928 - categorical_accuracy: 0.3670 - val_loss: 2.1951 - val_categorical_accuracy: 0.2472
Epoch 6/6
609/609 [==============================] - 185s 304ms/step - loss: 1.5278 - categorical_accuracy: 0.3840 - val_loss: 2.1691 - val_categorical_accuracy: 0.2

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model_3 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

609/609 [==============================] - 206s 309ms/step - loss: 2.0328 - categorical_accuracy: 0.2466 - val_loss: 2.2216 - val_categorical_accuracy: 0.2224
Epoch 2/6
609/609 [==============================] - 186s 305ms/step - loss: 2.0144 - categorical_accuracy: 0.2596 - val_loss: 2.2084 - val_categorical_accuracy: 0.2169
Epoch 3/6
609/609 [==============================] - 186s 305ms/step - loss: 1.9834 - categorical_accuracy: 0.2622 - val_loss: 2.2119 - val_categorical_accuracy: 0.2215
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 1.9636 - categorical_accuracy: 0.2674 - val_loss: 2.2154 - val_categorical_accuracy: 0.2206
              precision    recall  f1-score   support

         0.0       0.14      0.23      0.18       190
         1.0       0.16      0.32      0.21       173
         2.0       0.07      0.10      0.08       135
         3.0       0.12      0.08      0.10       216
         4.0       0.71      0.17      0.27      1036
         

1 H 44 min

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.2
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = bert_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    print(model1.summary())

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.2


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model_4 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

609/609 [==============================] - 205s 309ms/step - loss: 1.7462 - categorical_accuracy: 0.3286 - val_loss: 2.0768 - val_categorical_accuracy: 0.2160
Epoch 2/6
609/609 [==============================] - 185s 304ms/step - loss: 1.6618 - categorical_accuracy: 0.3458 - val_loss: 1.9819 - val_categorical_accuracy: 0.2454
Epoch 3/6
609/609 [==============================] - 185s 304ms/step - loss: 1.5568 - categorical_accuracy: 0.3698 - val_loss: 2.0413 - val_categorical_accuracy: 0.2426
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 1.4737 - categorical_accuracy: 0.3970 - val_loss: 2.0970 - val_categorical_accuracy: 0.2270
Epoch 5/6
609/609 [==============================] - 185s 305ms/step - loss: 1.3981 - categorical_accuracy: 0.4135 - val_loss: 2.0257 - val_categorical_accuracy: 0.2381
              precision    recall  f1-score   support

         0.0       0.15      0.36      0.21       190
         1.0       0.20      0.34      0.25       173
  

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


...

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model_6 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

609/609 [==============================] - 204s 310ms/step - loss: 2.1077 - categorical_accuracy: 0.2273 - val_loss: 2.2259 - val_categorical_accuracy: 0.1866
Epoch 2/6
609/609 [==============================] - 186s 305ms/step - loss: 2.0115 - categorical_accuracy: 0.2469 - val_loss: 2.2951 - val_categorical_accuracy: 0.1682
Epoch 3/6
609/609 [==============================] - 186s 305ms/step - loss: 1.9194 - categorical_accuracy: 0.2677 - val_loss: 2.1376 - val_categorical_accuracy: 0.2022
Epoch 4/6
609/609 [==============================] - 186s 305ms/step - loss: 1.8540 - categorical_accuracy: 0.2891 - val_loss: 2.2112 - val_categorical_accuracy: 0.1774
Epoch 5/6
609/609 [==============================] - 186s 305ms/step - loss: 1.7846 - categorical_accuracy: 0.3028 - val_loss: 2.1609 - val_categorical_accuracy: 0.1949
Epoch 6/6
609/609 [==============================] - 186s 305ms/step - loss: 1.7195 - categorical_accuracy: 0.3129 - val_loss: 2.2028 - val_categorical_accuracy: 0.2

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model_7 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

609/609 [==============================] - 205s 310ms/step - loss: 2.1046 - categorical_accuracy: 0.2474 - val_loss: 2.2138 - val_categorical_accuracy: 0.2270
Epoch 2/6
609/609 [==============================] - 186s 305ms/step - loss: 2.0697 - categorical_accuracy: 0.2546 - val_loss: 2.1917 - val_categorical_accuracy: 0.2261
Epoch 3/6
609/609 [==============================] - 186s 305ms/step - loss: 2.0566 - categorical_accuracy: 0.2596 - val_loss: 2.1867 - val_categorical_accuracy: 0.2362
Epoch 4/6
609/609 [==============================] - 186s 306ms/step - loss: 2.0326 - categorical_accuracy: 0.2617 - val_loss: 2.1664 - val_categorical_accuracy: 0.2390
Epoch 5/6
609/609 [==============================] - 186s 305ms/step - loss: 2.0109 - categorical_accuracy: 0.2646 - val_loss: 2.1360 - val_categorical_accuracy: 0.2472
Epoch 6/6
609/609 [==============================] - 186s 305ms/step - loss: 2.0128 - categorical_accuracy: 0.2675 - val_loss: 2.1535 - val_categorical_accuracy: 0.2

In [ ]:
balanced_accuracies_fine_tuning

[0.3115620755189979,
 0.2872598061998872,
 0.2904218211763925,
 0.2907618196948229]

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.3
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = bert_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    print(model1.summary())

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.3


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model_8 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

609/609 [==============================] - 208s 310ms/step - loss: 1.8343 - categorical_accuracy: 0.2994 - val_loss: 2.2037 - val_categorical_accuracy: 0.1994
Epoch 2/6
609/609 [==============================] - 186s 305ms/step - loss: 1.7307 - categorical_accuracy: 0.3169 - val_loss: 1.9064 - val_categorical_accuracy: 0.2619
Epoch 3/6
609/609 [==============================] - 186s 305ms/step - loss: 1.6662 - categorical_accuracy: 0.3398 - val_loss: 2.0382 - val_categorical_accuracy: 0.2279
Epoch 4/6
609/609 [==============================] - 186s 305ms/step - loss: 1.5915 - categorical_accuracy: 0.3558 - val_loss: 2.0383 - val_categorical_accuracy: 0.2215
Epoch 5/6
609/609 [==============================] - 186s 305ms/step - loss: 1.5239 - categorical_accuracy: 0.3779 - val_loss: 1.9862 - val_categorical_accuracy: 0.2362
              precision    recall  f1-score   support

         0.0       0.18      0.34      0.23       190
         1.0       0.20      0.50      0.29       173
  

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


...

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model_10 (TFBertModel)  TFBaseModelOutputWi  108310272  ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                        

609/609 [==============================] - 207s 310ms/step - loss: 2.1536 - categorical_accuracy: 0.2260 - val_loss: 2.4214 - val_categorical_accuracy: 0.1517
Epoch 2/6
609/609 [==============================] - 186s 305ms/step - loss: 2.0548 - categorical_accuracy: 0.2438 - val_loss: 2.3240 - val_categorical_accuracy: 0.1737
Epoch 3/6
609/609 [==============================] - 186s 305ms/step - loss: 1.9990 - categorical_accuracy: 0.2594 - val_loss: 2.3548 - val_categorical_accuracy: 0.1847
Epoch 4/6
609/609 [==============================] - 186s 305ms/step - loss: 1.9510 - categorical_accuracy: 0.2684 - val_loss: 2.3624 - val_categorical_accuracy: 0.1857
Epoch 5/6
609/609 [==============================] - 186s 305ms/step - loss: 1.8778 - categorical_accuracy: 0.2891 - val_loss: 2.4155 - val_categorical_accuracy: 0.1801
Epoch 6/6
609/609 [==============================] - 186s 305ms/step - loss: 1.8361 - categorical_accuracy: 0.2968 - val_loss: 2.1882 - val_categorical_accuracy: 0.2

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model_11 (TFBertModel)  TFBaseModelOutputWi  108310272  ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                        

609/609 [==============================] - 205s 310ms/step - loss: 2.1899 - categorical_accuracy: 0.2310 - val_loss: 2.2576 - val_categorical_accuracy: 0.2004
Epoch 2/6
609/609 [==============================] - 185s 304ms/step - loss: 2.1636 - categorical_accuracy: 0.2354 - val_loss: 2.2333 - val_categorical_accuracy: 0.1985
Epoch 3/6
609/609 [==============================] - 185s 304ms/step - loss: 2.1462 - categorical_accuracy: 0.2406 - val_loss: 2.2322 - val_categorical_accuracy: 0.1985
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 2.1242 - categorical_accuracy: 0.2385 - val_loss: 2.2225 - val_categorical_accuracy: 0.2040
Epoch 5/6
609/609 [==============================] - 185s 304ms/step - loss: 2.0941 - categorical_accuracy: 0.2464 - val_loss: 2.2332 - val_categorical_accuracy: 0.1939
Epoch 6/6
609/609 [==============================] - 185s 304ms/step - loss: 2.0884 - categorical_accuracy: 0.2530 - val_loss: 2.1772 - val_categorical_accuracy: 0.2

In [ ]:
balanced_accuracies_fine_tuning

[0.2978907996233228,
 0.3072547196607738,
 0.33623967038181124,
 0.2984879713938942]

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.4
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = bert_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    print(model1.summary())

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.4


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model_12 (TFBertModel)  TFBaseModelOutputWi  108310272  ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                        

609/609 [==============================] - 206s 309ms/step - loss: 1.8742 - categorical_accuracy: 0.2843 - val_loss: 2.0269 - val_categorical_accuracy: 0.2086
Epoch 2/6
609/609 [==============================] - 185s 305ms/step - loss: 1.7724 - categorical_accuracy: 0.3074 - val_loss: 1.9860 - val_categorical_accuracy: 0.2555
Epoch 3/6
609/609 [==============================] - 185s 304ms/step - loss: 1.7088 - categorical_accuracy: 0.3216 - val_loss: 2.0104 - val_categorical_accuracy: 0.2252
Epoch 4/6
609/609 [==============================] - 186s 305ms/step - loss: 1.6375 - categorical_accuracy: 0.3403 - val_loss: 1.9102 - val_categorical_accuracy: 0.2463
Epoch 5/6
609/609 [==============================] - 186s 305ms/step - loss: 1.5897 - categorical_accuracy: 0.3639 - val_loss: 2.0252 - val_categorical_accuracy: 0.2647
Epoch 6/6
609/609 [==============================] - 186s 305ms/step - loss: 1.5456 - categorical_accuracy: 0.3622 - val_loss: 2.0124 - val_categorical_accuracy: 0.2

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


...

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model_14 (TFBertModel)  TFBaseModelOutputWi  108310272  ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                        

609/609 [==============================] - 205s 311ms/step - loss: 2.2454 - categorical_accuracy: 0.2037 - val_loss: 2.3167 - val_categorical_accuracy: 0.1517
Epoch 2/6
609/609 [==============================] - 185s 304ms/step - loss: 2.1576 - categorical_accuracy: 0.2081 - val_loss: 2.2810 - val_categorical_accuracy: 0.1572
Epoch 3/6
609/609 [==============================] - 186s 305ms/step - loss: 2.0999 - categorical_accuracy: 0.2238 - val_loss: 2.0879 - val_categorical_accuracy: 0.1958
Epoch 4/6
609/609 [==============================] - 185s 305ms/step - loss: 2.0538 - categorical_accuracy: 0.2342 - val_loss: 2.1356 - val_categorical_accuracy: 0.1884
Epoch 5/6
609/609 [==============================] - 185s 304ms/step - loss: 1.9923 - categorical_accuracy: 0.2401 - val_loss: 2.0899 - val_categorical_accuracy: 0.2059
Epoch 6/6
609/609 [==============================] - 185s 305ms/step - loss: 1.9590 - categorical_accuracy: 0.2514 - val_loss: 1.9926 - val_categorical_accuracy: 0.2

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model_15 (TFBertModel)  TFBaseModelOutputWi  108310272  ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                        

609/609 [==============================] - 206s 309ms/step - loss: 2.2480 - categorical_accuracy: 0.2106 - val_loss: 2.1989 - val_categorical_accuracy: 0.1866
Epoch 2/6
609/609 [==============================] - 186s 305ms/step - loss: 2.2344 - categorical_accuracy: 0.2115 - val_loss: 2.2070 - val_categorical_accuracy: 0.1958
Epoch 3/6
609/609 [==============================] - 186s 305ms/step - loss: 2.1976 - categorical_accuracy: 0.2133 - val_loss: 2.1671 - val_categorical_accuracy: 0.2215
Epoch 4/6
609/609 [==============================] - 186s 305ms/step - loss: 2.2025 - categorical_accuracy: 0.2226 - val_loss: 2.1564 - val_categorical_accuracy: 0.2151
Epoch 5/6
609/609 [==============================] - 185s 304ms/step - loss: 2.1878 - categorical_accuracy: 0.2148 - val_loss: 2.1272 - val_categorical_accuracy: 0.2390
Epoch 6/6
609/609 [==============================] - 185s 304ms/step - loss: 2.1710 - categorical_accuracy: 0.2250 - val_loss: 2.1497 - val_categorical_accuracy: 0.2

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.5
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = bert_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)#added
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    print(model1.summary())

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.5


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model_16 (TFBertModel)  TFBaseModelOutputWi  108310272  ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                        

609/609 [==============================] - 205s 309ms/step - loss: 1.9112 - categorical_accuracy: 0.2781 - val_loss: 2.1209 - val_categorical_accuracy: 0.1829
Epoch 2/6
609/609 [==============================] - 185s 304ms/step - loss: 1.8513 - categorical_accuracy: 0.2934 - val_loss: 2.0689 - val_categorical_accuracy: 0.2261
Epoch 3/6
609/609 [==============================] - 185s 304ms/step - loss: 1.7980 - categorical_accuracy: 0.3018 - val_loss: 1.9970 - val_categorical_accuracy: 0.2325
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 1.7573 - categorical_accuracy: 0.3095 - val_loss: 1.9560 - val_categorical_accuracy: 0.2224
Epoch 5/6
609/609 [==============================] - 185s 304ms/step - loss: 1.7049 - categorical_accuracy: 0.3346 - val_loss: 2.0359 - val_categorical_accuracy: 0.2482
Epoch 6/6
609/609 [==============================] - 185s 304ms/step - loss: 1.6667 - categorical_accuracy: 0.3391 - val_loss: 1.9463 - val_categorical_accuracy: 0.2

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


...

Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model_18 (TFBertModel)  TFBaseModelOutputWi  108310272  ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                        

609/609 [==============================] - 206s 309ms/step - loss: 2.3584 - categorical_accuracy: 0.1923 - val_loss: 2.2391 - val_categorical_accuracy: 0.1682
Epoch 2/6
609/609 [==============================] - 186s 305ms/step - loss: 2.2737 - categorical_accuracy: 0.1996 - val_loss: 2.4010 - val_categorical_accuracy: 0.1654
Epoch 3/6
609/609 [==============================] - 185s 305ms/step - loss: 2.2404 - categorical_accuracy: 0.2072 - val_loss: 2.4347 - val_categorical_accuracy: 0.1599
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 2.1597 - categorical_accuracy: 0.2181 - val_loss: 2.4060 - val_categorical_accuracy: 0.1719
Epoch 5/6
609/609 [==============================] - 186s 305ms/step - loss: 2.1290 - categorical_accuracy: 0.2300 - val_loss: 2.5178 - val_categorical_accuracy: 0.1562
Epoch 6/6
609/609 [==============================] - 186s 305ms/step - loss: 2.1129 - categorical_accuracy: 0.2258 - val_loss: 2.0889 - val_categorical_accuracy: 0.2

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model_19 (TFBertModel)  TFBaseModelOutputWi  108310272  ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                        

609/609 [==============================] - 206s 309ms/step - loss: 2.3864 - categorical_accuracy: 0.1940 - val_loss: 2.3040 - val_categorical_accuracy: 0.1581
Epoch 2/6
609/609 [==============================] - 186s 305ms/step - loss: 2.3871 - categorical_accuracy: 0.1925 - val_loss: 2.2908 - val_categorical_accuracy: 0.1553
Epoch 3/6
609/609 [==============================] - 186s 305ms/step - loss: 2.3389 - categorical_accuracy: 0.2001 - val_loss: 2.2656 - val_categorical_accuracy: 0.1553
Epoch 4/6
609/609 [==============================] - 186s 305ms/step - loss: 2.3403 - categorical_accuracy: 0.2043 - val_loss: 2.2647 - val_categorical_accuracy: 0.1590
Epoch 5/6
609/609 [==============================] - 186s 305ms/step - loss: 2.3109 - categorical_accuracy: 0.2046 - val_loss: 2.2260 - val_categorical_accuracy: 0.1765
Epoch 6/6
609/609 [==============================] - 186s 305ms/step - loss: 2.3329 - categorical_accuracy: 0.2005 - val_loss: 2.2375 - val_categorical_accuracy: 0.1

In [ ]:
McNemar_BERT_batch_normalize_and_dropout_split0 =pd.DataFrame(data=McNemar[(True, 0.3,1e-3,1e-5)])
McNemar_BERT_batch_normalize_and_dropout_split0.to_csv(DIR + 'McNemar_BERT_batch_normalize_and_dropout_split0_fine_tune.csv')

McNemar_BERT_batch_normalize_and_dropout_split0 =pd.DataFrame(data=McNemar[(False, 0.3,1e-3,1e-5)])
McNemar_BERT_batch_normalize_and_dropout_split0.to_csv(DIR + 'McNemar_BERT_batch_normalize_and_dropout_split0_transfer_learning.csv')

### Resume from the last cell

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.3
learning_rate_transfer_learnings = [ 1e-4]
learning_rate_fine_tunings = [1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = bert_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    #model2.summary()

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.0001 learning_rate_fine_tuning:  1e-06 drop_out_rate: 0.3


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 95s 135ms/step - loss: 2.4495 - categorical_accuracy: 0.1297 - val_loss: 2.3345 - val_categorical_accuracy: 0.1406
Epoch 2/6
609/609 [==============================] - 79s 130ms/step - loss: 2.2148 - categorical_accuracy: 0.1918 - val_loss: 2.2765 - val_categorical_accuracy: 0.1691
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 2.1319 - categorical_accuracy: 0.2072 - val_loss: 2.2452 - val_categorical_accuracy: 0.1765
Epoch 4/6
609/609 [==============================] - 79s 130ms/step - loss: 2.0613 - categorical_accuracy: 0.2320 - val_loss: 2.2178 - val_categorical_accuracy: 0.1921
Epoch 5/6
609/609 [==============================] - 80s 132ms/step - loss: 2.0293 - categorical_accuracy: 0.2409 - val_loss: 2.1922 - val_categorical_accuracy: 0.2022
Epoch 6/6
609/609 [==============================] - 80s 131ms/step - loss: 1.9828 - categorical_accuracy: 0.2560 - val_loss: 2.1917 - val_categorical_accuracy:

609/609 [==============================] - 204s 309ms/step - loss: 1.9627 - categorical_accuracy: 0.2607 - val_loss: 2.1839 - val_categorical_accuracy: 0.1893
Epoch 2/6
609/609 [==============================] - 186s 305ms/step - loss: 1.9341 - categorical_accuracy: 0.2688 - val_loss: 2.1393 - val_categorical_accuracy: 0.2123
Epoch 3/6
609/609 [==============================] - 186s 305ms/step - loss: 1.9075 - categorical_accuracy: 0.2677 - val_loss: 2.1315 - val_categorical_accuracy: 0.2123
Epoch 4/6
609/609 [==============================] - 186s 306ms/step - loss: 1.8853 - categorical_accuracy: 0.2781 - val_loss: 2.1393 - val_categorical_accuracy: 0.2086
Epoch 5/6
609/609 [==============================] - 186s 306ms/step - loss: 1.8665 - categorical_accuracy: 0.2827 - val_loss: 2.1286 - val_categorical_accuracy: 0.2178
Epoch 6/6
609/609 [==============================] - 186s 305ms/step - loss: 1.8527 - categorical_accuracy: 0.2840 - val_loss: 2.1103 - val_categorical_accuracy: 0.2

In [ ]:
seen_parameter = []#(drop_out_rate, initial_rate, fine_tune_learning_rate)

In [ ]:
McNemar={}
balanced_accuracies_transfer_learning=[]
balanced_accuracies_fine_tuning = []

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.4
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = bert_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    #model2.summary()

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.4


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 134ms/step - loss: 2.2317 - categorical_accuracy: 0.1972 - val_loss: 2.1380 - val_categorical_accuracy: 0.1912
Epoch 2/6
609/609 [==============================] - 79s 129ms/step - loss: 2.0176 - categorical_accuracy: 0.2512 - val_loss: 2.1589 - val_categorical_accuracy: 0.2086
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9597 - categorical_accuracy: 0.2723 - val_loss: 2.1580 - val_categorical_accuracy: 0.1930
Epoch 4/6
609/609 [==============================] - 79s 129ms/step - loss: 1.9069 - categorical_accuracy: 0.2855 - val_loss: 2.1173 - val_categorical_accuracy: 0.2491
Epoch 5/6
609/609 [==============================] - 79s 129ms/step - loss: 1.8727 - categorical_accuracy: 0.2930 - val_loss: 2.0645 - val_categorical_accuracy: 0.2408
Epoch 6/6
609/609 [==============================] - 79s 130ms/step - loss: 1.8497 - categorical_accuracy: 0.3043 - val_loss: 2.0751 - val_categorical_accuracy:

609/609 [==============================] - 202s 305ms/step - loss: 1.8155 - categorical_accuracy: 0.2956 - val_loss: 2.0693 - val_categorical_accuracy: 0.2417
Epoch 2/6
609/609 [==============================] - 183s 301ms/step - loss: 1.6760 - categorical_accuracy: 0.3289 - val_loss: 2.0994 - val_categorical_accuracy: 0.2463
Epoch 3/6
609/609 [==============================] - 183s 301ms/step - loss: 1.5883 - categorical_accuracy: 0.3566 - val_loss: 1.9835 - val_categorical_accuracy: 0.2840
Epoch 4/6
609/609 [==============================] - 183s 301ms/step - loss: 1.5029 - categorical_accuracy: 0.3783 - val_loss: 1.9055 - val_categorical_accuracy: 0.2987
Epoch 5/6
609/609 [==============================] - 183s 301ms/step - loss: 1.4270 - categorical_accuracy: 0.4063 - val_loss: 1.9359 - val_categorical_accuracy: 0.3107
Epoch 6/6
609/609 [==============================] - 183s 301ms/step - loss: 1.3398 - categorical_accuracy: 0.4276 - val_loss: 1.8814 - val_categorical_accuracy: 0.3

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.2068 - categorical_accuracy: 0.2057 - val_loss: 2.2205 - val_categorical_accuracy: 0.1461
Epoch 2/6
609/609 [==============================] - 79s 130ms/step - loss: 2.0150 - categorical_accuracy: 0.2399 - val_loss: 2.1568 - val_categorical_accuracy: 0.1930
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9432 - categorical_accuracy: 0.2668 - val_loss: 2.1433 - val_categorical_accuracy: 0.2123
Epoch 4/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9005 - categorical_accuracy: 0.2740 - val_loss: 2.0847 - val_categorical_accuracy: 0.2132
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 1.8612 - categorical_accuracy: 0.2830 - val_loss: 2.1234 - val_categorical_accuracy: 0.2123
Epoch 6/6
609/609 [==============================] - 80s 131ms/step - loss: 1.8466 - categorical_accuracy: 0.2885 - val_loss: 2.1262 - val_categorical_accuracy:

609/609 [==============================] - 202s 306ms/step - loss: 1.7734 - categorical_accuracy: 0.3100 - val_loss: 2.1188 - val_categorical_accuracy: 0.2142
Epoch 2/6
609/609 [==============================] - 183s 301ms/step - loss: 1.7511 - categorical_accuracy: 0.3136 - val_loss: 2.0763 - val_categorical_accuracy: 0.2371
Epoch 3/6
609/609 [==============================] - 183s 301ms/step - loss: 1.7396 - categorical_accuracy: 0.3172 - val_loss: 2.0898 - val_categorical_accuracy: 0.2233
Epoch 4/6
609/609 [==============================] - 183s 301ms/step - loss: 1.7041 - categorical_accuracy: 0.3252 - val_loss: 2.0878 - val_categorical_accuracy: 0.2353
Epoch 5/6
609/609 [==============================] - 183s 301ms/step - loss: 1.6946 - categorical_accuracy: 0.3273 - val_loss: 2.0737 - val_categorical_accuracy: 0.2261
              precision    recall  f1-score   support

         0.0       0.11      0.21      0.15       190
         1.0       0.15      0.58      0.24       173
  

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.4721 - categorical_accuracy: 0.1269 - val_loss: 2.3431 - val_categorical_accuracy: 0.1369
Epoch 2/6
609/609 [==============================] - 79s 130ms/step - loss: 2.2818 - categorical_accuracy: 0.1688 - val_loss: 2.2817 - val_categorical_accuracy: 0.1452
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 2.1810 - categorical_accuracy: 0.1978 - val_loss: 2.2541 - val_categorical_accuracy: 0.1535
Epoch 4/6
609/609 [==============================] - 79s 130ms/step - loss: 2.1360 - categorical_accuracy: 0.2097 - val_loss: 2.2206 - val_categorical_accuracy: 0.1645
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 2.0921 - categorical_accuracy: 0.2238 - val_loss: 2.1995 - val_categorical_accuracy: 0.1728
Epoch 6/6
609/609 [==============================] - 79s 130ms/step - loss: 2.0520 - categorical_accuracy: 0.2474 - val_loss: 2.1727 - val_categorical_accuracy:

609/609 [==============================] - 202s 305ms/step - loss: 1.9919 - categorical_accuracy: 0.2618 - val_loss: 1.9537 - val_categorical_accuracy: 0.2895
Epoch 2/6
609/609 [==============================] - 183s 301ms/step - loss: 1.8824 - categorical_accuracy: 0.2885 - val_loss: 2.0117 - val_categorical_accuracy: 0.2675
Epoch 3/6
392/609 [==================>...........] - ETA: 1:02 - loss: 1.7924 - categorical_accuracy: 0.3181

Resume from the last

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.4
learning_rate_transfer_learnings = [1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = bert_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    #model2.summary()

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.0001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.4


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 96s 136ms/step - loss: 2.5474 - categorical_accuracy: 0.1922 - val_loss: 2.2609 - val_categorical_accuracy: 0.1507
Epoch 2/6
609/609 [==============================] - 80s 131ms/step - loss: 2.2904 - categorical_accuracy: 0.1937 - val_loss: 2.2255 - val_categorical_accuracy: 0.1627
Epoch 3/6
609/609 [==============================] - 80s 131ms/step - loss: 2.1950 - categorical_accuracy: 0.2088 - val_loss: 2.2120 - val_categorical_accuracy: 0.1756
Epoch 4/6
609/609 [==============================] - 80s 131ms/step - loss: 2.1501 - categorical_accuracy: 0.2247 - val_loss: 2.1856 - val_categorical_accuracy: 0.1710
Epoch 5/6
609/609 [==============================] - 80s 131ms/step - loss: 2.0989 - categorical_accuracy: 0.2344 - val_loss: 2.1620 - val_categorical_accuracy: 0.1838
Epoch 6/6
609/609 [==============================] - 80s 131ms/step - loss: 2.0665 - categorical_accuracy: 0.2503 - val_loss: 2.1514 - val_categorical_accuracy:

609/609 [==============================] - 205s 308ms/step - loss: 2.0187 - categorical_accuracy: 0.2574 - val_loss: 2.1205 - val_categorical_accuracy: 0.2096
Epoch 2/6
609/609 [==============================] - 185s 304ms/step - loss: 1.9338 - categorical_accuracy: 0.2912 - val_loss: 2.0420 - val_categorical_accuracy: 0.2537
Epoch 3/6
609/609 [==============================] - 185s 304ms/step - loss: 1.8401 - categorical_accuracy: 0.3116 - val_loss: 2.1483 - val_categorical_accuracy: 0.2233
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 1.7726 - categorical_accuracy: 0.3328 - val_loss: 2.0875 - val_categorical_accuracy: 0.2482
Epoch 5/6
609/609 [==============================] - 185s 304ms/step - loss: 1.6934 - categorical_accuracy: 0.3549 - val_loss: 2.0820 - val_categorical_accuracy: 0.2472
              precision    recall  f1-score   support

         0.0       0.14      0.28      0.19       190
         1.0       0.17      0.44      0.24       173
  

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.4859 - categorical_accuracy: 0.1369 - val_loss: 2.2595 - val_categorical_accuracy: 0.1664
Epoch 2/6
609/609 [==============================] - 79s 130ms/step - loss: 2.2612 - categorical_accuracy: 0.1911 - val_loss: 2.2091 - val_categorical_accuracy: 0.1783
Epoch 3/6
609/609 [==============================] - 80s 131ms/step - loss: 2.1590 - categorical_accuracy: 0.2080 - val_loss: 2.1692 - val_categorical_accuracy: 0.1912
Epoch 4/6
609/609 [==============================] - 80s 131ms/step - loss: 2.1282 - categorical_accuracy: 0.2221 - val_loss: 2.1616 - val_categorical_accuracy: 0.1875
Epoch 5/6
609/609 [==============================] - 80s 131ms/step - loss: 2.0859 - categorical_accuracy: 0.2359 - val_loss: 2.1383 - val_categorical_accuracy: 0.1958
Epoch 6/6
609/609 [==============================] - 80s 131ms/step - loss: 2.0407 - categorical_accuracy: 0.2432 - val_loss: 2.1390 - val_categorical_accuracy:

609/609 [==============================] - 205s 308ms/step - loss: 2.0157 - categorical_accuracy: 0.2485 - val_loss: 2.1258 - val_categorical_accuracy: 0.2151
Epoch 2/6
609/609 [==============================] - 185s 303ms/step - loss: 1.9896 - categorical_accuracy: 0.2572 - val_loss: 2.1042 - val_categorical_accuracy: 0.2096
Epoch 3/6
609/609 [==============================] - 185s 303ms/step - loss: 1.9563 - categorical_accuracy: 0.2642 - val_loss: 2.0781 - val_categorical_accuracy: 0.2233
Epoch 4/6
609/609 [==============================] - 185s 303ms/step - loss: 1.9456 - categorical_accuracy: 0.2640 - val_loss: 2.1164 - val_categorical_accuracy: 0.2224
Epoch 5/6
609/609 [==============================] - 185s 303ms/step - loss: 1.9211 - categorical_accuracy: 0.2731 - val_loss: 2.0909 - val_categorical_accuracy: 0.2381
Epoch 6/6
609/609 [==============================] - 185s 303ms/step - loss: 1.9165 - categorical_accuracy: 0.2787 - val_loss: 2.0890 - val_categorical_accuracy: 0.2

In [ ]:
from transformers import TFAutoModel


#drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256
#for drop_out_rate in drop_out_rates:

drop_out_rate = 0.5
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]

for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
    , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
    #step1
    bert_base = TFAutoModel.from_pretrained('bert-base-cased')
    input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

    embeddings = bert_base(input_ids, attention_mask= mask)[0]
    X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Dense(128, activation = 'relu')(X)
    X = tf.keras.layers.Dropout(drop_out_rate)(X)
    X = tf.keras.layers.Dense(32, activation = 'relu')(X)
    y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

    model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    model1.layers[2].trainable = False
    #model2.summary()

    #step2
    optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
    #model2.summary() #Check trainable params increased.

    #step3: transfer learning
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

    #step4: predict
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_transfer_learning.append( balanced_acc )

    #step5: fine tune
    print("Fine tuning---------------")
    model1.layers[2].trainable = True

    # It's important to recompile your model after you make any changes
    optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
    loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
    metrics = []
    metrics.append(
        tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
    model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

    history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
    balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
    balanced_accuracies_fine_tuning.append( balanced_acc )
    print("----------------------------------------")
    del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.5


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.2785 - categorical_accuracy: 0.1874 - val_loss: 2.1913 - val_categorical_accuracy: 0.1756
Epoch 2/6
609/609 [==============================] - 80s 131ms/step - loss: 2.0369 - categorical_accuracy: 0.2328 - val_loss: 2.1467 - val_categorical_accuracy: 0.2013
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9804 - categorical_accuracy: 0.2607 - val_loss: 2.0973 - val_categorical_accuracy: 0.2243
Epoch 4/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9389 - categorical_accuracy: 0.2745 - val_loss: 2.0932 - val_categorical_accuracy: 0.2472
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 1.9070 - categorical_accuracy: 0.2903 - val_loss: 2.0543 - val_categorical_accuracy: 0.2482
Epoch 6/6
609/609 [==============================] - 80s 131ms/step - loss: 1.8789 - categorical_accuracy: 0.2850 - val_loss: 2.0647 - val_categorical_accuracy:

609/609 [==============================] - 205s 308ms/step - loss: 1.8338 - categorical_accuracy: 0.3029 - val_loss: 1.9680 - val_categorical_accuracy: 0.2619
Epoch 2/6
609/609 [==============================] - 185s 304ms/step - loss: 1.7193 - categorical_accuracy: 0.3211 - val_loss: 1.9922 - val_categorical_accuracy: 0.2656
Epoch 3/6
609/609 [==============================] - 185s 304ms/step - loss: 1.6199 - categorical_accuracy: 0.3390 - val_loss: 1.9621 - val_categorical_accuracy: 0.2574
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 1.5442 - categorical_accuracy: 0.3506 - val_loss: 1.9326 - val_categorical_accuracy: 0.2739
Epoch 5/6
609/609 [==============================] - 185s 304ms/step - loss: 1.4593 - categorical_accuracy: 0.3778 - val_loss: 1.9253 - val_categorical_accuracy: 0.3015
Epoch 6/6
609/609 [==============================] - 185s 304ms/step - loss: 1.3994 - categorical_accuracy: 0.3975 - val_loss: 1.8392 - val_categorical_accuracy: 0.3

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


...

Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.5205 - categorical_accuracy: 0.1425 - val_loss: 2.3374 - val_categorical_accuracy: 0.1480
Epoch 2/6
609/609 [==============================] - 79s 130ms/step - loss: 2.3057 - categorical_accuracy: 0.1819 - val_loss: 2.2816 - val_categorical_accuracy: 0.1664
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 2.2312 - categorical_accuracy: 0.2128 - val_loss: 2.2311 - val_categorical_accuracy: 0.1811
Epoch 4/6
609/609 [==============================] - 79s 130ms/step - loss: 2.1599 - categorical_accuracy: 0.2236 - val_loss: 2.2317 - val_categorical_accuracy: 0.1866
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 2.1126 - categorical_accuracy: 0.2339 - val_loss: 2.2113 - val_categorical_accuracy: 0.1847
Epoch 6/6
609/609 [==============================] - 79s 130ms/step - loss: 2.0773 - categorical_accuracy: 0.2416 - val_loss: 2.1986 - val_categorical_accuracy:

609/609 [==============================] - 204s 308ms/step - loss: 2.0316 - categorical_accuracy: 0.2578 - val_loss: 2.2454 - val_categorical_accuracy: 0.1811
Epoch 2/6
609/609 [==============================] - 185s 304ms/step - loss: 1.9390 - categorical_accuracy: 0.2811 - val_loss: 1.9728 - val_categorical_accuracy: 0.2969
Epoch 3/6
609/609 [==============================] - 185s 304ms/step - loss: 1.8722 - categorical_accuracy: 0.3072 - val_loss: 2.1670 - val_categorical_accuracy: 0.2289
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 1.8112 - categorical_accuracy: 0.3215 - val_loss: 2.1646 - val_categorical_accuracy: 0.2335
Epoch 5/6
609/609 [==============================] - 185s 304ms/step - loss: 1.7418 - categorical_accuracy: 0.3276 - val_loss: 2.0594 - val_categorical_accuracy: 0.2757
              precision    recall  f1-score   support

         0.0       0.14      0.19      0.16       190
         1.0       0.19      0.50      0.28       173
  

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 94s 135ms/step - loss: 2.5813 - categorical_accuracy: 0.1196 - val_loss: 2.2888 - val_categorical_accuracy: 0.1287
Epoch 2/6
609/609 [==============================] - 79s 131ms/step - loss: 2.3399 - categorical_accuracy: 0.1607 - val_loss: 2.2221 - val_categorical_accuracy: 0.1535
Epoch 3/6
609/609 [==============================] - 79s 130ms/step - loss: 2.2385 - categorical_accuracy: 0.1825 - val_loss: 2.1884 - val_categorical_accuracy: 0.1700
Epoch 4/6
609/609 [==============================] - 80s 131ms/step - loss: 2.1680 - categorical_accuracy: 0.2026 - val_loss: 2.1762 - val_categorical_accuracy: 0.1765
Epoch 5/6
609/609 [==============================] - 79s 130ms/step - loss: 2.1307 - categorical_accuracy: 0.2115 - val_loss: 2.1658 - val_categorical_accuracy: 0.1829
Epoch 6/6
609/609 [==============================] - 80s 131ms/step - loss: 2.1001 - categorical_accuracy: 0.2184 - val_loss: 2.1603 - val_categorical_accuracy:

609/609 [==============================] - 205s 308ms/step - loss: 2.0619 - categorical_accuracy: 0.2253 - val_loss: 2.1327 - val_categorical_accuracy: 0.1939
Epoch 2/6
609/609 [==============================] - 185s 304ms/step - loss: 2.0486 - categorical_accuracy: 0.2334 - val_loss: 2.1031 - val_categorical_accuracy: 0.1967
Epoch 3/6
609/609 [==============================] - 185s 304ms/step - loss: 2.0223 - categorical_accuracy: 0.2395 - val_loss: 2.1161 - val_categorical_accuracy: 0.1903
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 2.0064 - categorical_accuracy: 0.2426 - val_loss: 2.0986 - val_categorical_accuracy: 0.2013
Epoch 5/6
609/609 [==============================] - 185s 304ms/step - loss: 1.9984 - categorical_accuracy: 0.2461 - val_loss: 2.1002 - val_categorical_accuracy: 0.1994
Epoch 6/6
609/609 [==============================] - 185s 304ms/step - loss: 1.9790 - categorical_accuracy: 0.2593 - val_loss: 2.0882 - val_categorical_accuracy: 0.2

In [ ]:
McNemar.keys()

dict_keys([(False, 0.4, 0.0001, 1e-05), (True, 0.4, 0.0001, 1e-05), (False, 0.4, 0.0001, 1e-06), (True, 0.4, 0.0001, 1e-06), (False, 0.5, 0.001, 1e-05), (True, 0.5, 0.001, 1e-05), (False, 0.5, 0.001, 1e-06), (True, 0.5, 0.001, 1e-06), (False, 0.5, 0.0001, 1e-05), (True, 0.5, 0.0001, 1e-05), (False, 0.5, 0.0001, 1e-06), (True, 0.5, 0.0001, 1e-06)])

In [ ]:
df_McNemar_BERT_split0_fine_tune = pd.DataFrame(data=McNemar[(True,0.5, 1e-3, 1e-5)])
df_McNemar_BERT_split0_fine_tune.to_csv(DIR + 'McNemar_BERT_split0_fine_tune.csv')

In [ ]:
df_McNemar_BERT_split0_transfer_learning = pd.DataFrame(data=McNemar[(False,0.5, 1e-3, 1e-5)])
df_McNemar_BERT_split0_transfer_learning.to_csv(DIR + 'McNemar_BERT_split0_transfer_learning.csv')

In [ ]:
balanced_accuracies_fine_tuning

[0.3545180413326312,
 0.32561995928565546,
 0.3198687138125366,
 0.2922621302487614]

In [ ]:
balanced_accuracies_fine_tuning

[0.3545180413326312,
 0.32561995928565546,
 0.3198687138125366,
 0.2922621302487614,
 0.35156416694413817,
 0.3320729042652105,
 0.3001390737921044]

### Best parameter is: drop out rate= 0.3, initial learn rate= 1e-3 , fine tune rate= 1e-5

### KFold test

Best parameter set is: initial learning rate=1e-3, fine tuning learning rate=1e-5, drop out rate=0.2 

In [ ]:
del(dataset_train)
del(dataset_test)
del(train)
del(val)
del(test)

#### Fold1: intentionally separate cells for check point purpose.

In [29]:
from transformers import AutoTokenizer
import numpy as np

split0=splits[1]
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
SEQ_LEN=256
print(split0['X_train'].shape, split0['X_test'].shape)

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']

print("Xids_train.shape, Xids_test.shape: ",Xids_train.shape, Xids_test.shape)

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)
dataset_train = dataset_train.shuffle(42).batch(16)

train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

(10832,) (2708,)
Xids_train.shape, Xids_test.shape:  (10832, 256) (2708, 256)


In [30]:
drop_out_rate = 0.3
learning_rate_transfer_learning =1e-3
learning_rate_fine_tuning = 1e-5

In [33]:
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
McNemar={}

In [34]:
from transformers import TFAutoModel



#balanced_accuracies_transfer_learning = []
#balanced_accuracies_fine_tuning = []
SEQ_LEN2=256


print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
, learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
#step1
bert_base = TFAutoModel.from_pretrained('bert-base-cased')
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = bert_base(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model1.layers[2].trainable = False
print(model1.summary())

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
#model2.summary() #Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_transfer_learning.append( balanced_acc )

#step5: fine tune
print("Fine tuning---------------")
model1.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_fine_tuning.append( balanced_acc )
print("----------------------------------------")
del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.3


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

609/609 [==============================] - 354s 548ms/step - loss: 1.7822 - categorical_accuracy: 0.3125 - val_loss: 2.0159 - val_categorical_accuracy: 0.2426
Epoch 2/6
609/609 [==============================] - 330s 543ms/step - loss: 1.6509 - categorical_accuracy: 0.3402 - val_loss: 1.9121 - val_categorical_accuracy: 0.2785
Epoch 3/6
609/609 [==============================] - 331s 543ms/step - loss: 1.5396 - categorical_accuracy: 0.3649 - val_loss: 1.9414 - val_categorical_accuracy: 0.2675
Epoch 4/6
609/609 [==============================] - 331s 543ms/step - loss: 1.4338 - categorical_accuracy: 0.4022 - val_loss: 1.9224 - val_categorical_accuracy: 0.3051
Epoch 5/6
609/609 [==============================] - 331s 543ms/step - loss: 1.3592 - categorical_accuracy: 0.4243 - val_loss: 1.9451 - val_categorical_accuracy: 0.2610
Epoch 6/6
609/609 [==============================] - 331s 543ms/step - loss: 1.2797 - categorical_accuracy: 0.4475 - val_loss: 1.8610 - val_categorical_accuracy: 0.3

#### Fold2: 

In [35]:
from transformers import AutoTokenizer
import numpy as np

split0=splits[2]
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
SEQ_LEN=256
print(split0['X_train'].shape, split0['X_test'].shape)

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']

print("Xids_train.shape, Xids_test.shape: ",Xids_train.shape, Xids_test.shape)

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)
dataset_train = dataset_train.shuffle(42).batch(16)

train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

(10832,) (2708,)
Xids_train.shape, Xids_test.shape:  (10832, 256) (2708, 256)


In [36]:
drop_out_rate = 0.3
learning_rate_transfer_learning =1e-3
learning_rate_fine_tuning = 1e-5

In [ ]:
from transformers import TFAutoModel



#balanced_accuracies_transfer_learning = []
#balanced_accuracies_fine_tuning = []
SEQ_LEN2=256


print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
, learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
#step1
bert_base = TFAutoModel.from_pretrained('bert-base-cased')
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = bert_base(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model1.layers[2].trainable = False
print(model1.summary())

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
#model2.summary() #Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_transfer_learning.append( balanced_acc )

#step5: fine tune
print("Fine tuning---------------")
model1.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_fine_tuning.append( balanced_acc )
print("----------------------------------------")
del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.3


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

#### Fold3: intentionally separate cells for check point purpose.

In [29]:
drop_out_rate = 0.3
learning_rate_transfer_learning =1e-3
learning_rate_fine_tuning = 1e-5

In [30]:
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
McNemar={}

In [31]:
from transformers import AutoTokenizer
import numpy as np

split0=splits[3]
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
SEQ_LEN=256
print(split0['X_train'].shape, split0['X_test'].shape)

#tokenizer = AutoTokenizer.from_pretrained('roberta-base')bert-base-cased
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']

print("Xids_train.shape, Xids_test.shape: ",Xids_train.shape, Xids_test.shape)

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)
dataset_train = dataset_train.shuffle(42).batch(16)

train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

(10832,) (2708,)
Xids_train.shape, Xids_test.shape:  (10832, 256) (2708, 256)


In [32]:
from transformers import TFAutoModel



#balanced_accuracies_transfer_learning = []
#balanced_accuracies_fine_tuning = []
SEQ_LEN2=256


print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
, learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
#step1
bert_base = TFAutoModel.from_pretrained('bert-base-cased')
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = bert_base(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model1.layers[2].trainable = False
print(model1.summary())

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
#model2.summary() #Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_transfer_learning.append( balanced_acc )

#step5: fine tune
print("Fine tuning---------------")
model1.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_fine_tuning.append( balanced_acc )
print("----------------------------------------")
del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.3


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

609/609 [==============================] - 203s 308ms/step - loss: 1.7695 - categorical_accuracy: 0.2835 - val_loss: 2.1862 - val_categorical_accuracy: 0.2050
Epoch 2/6
609/609 [==============================] - 184s 303ms/step - loss: 1.6028 - categorical_accuracy: 0.3236 - val_loss: 2.0537 - val_categorical_accuracy: 0.2335
Epoch 3/6
609/609 [==============================] - 184s 303ms/step - loss: 1.4946 - categorical_accuracy: 0.3557 - val_loss: 2.0228 - val_categorical_accuracy: 0.2509
Epoch 4/6
609/609 [==============================] - 184s 302ms/step - loss: 1.3826 - categorical_accuracy: 0.3920 - val_loss: 1.9838 - val_categorical_accuracy: 0.2601
Epoch 5/6
609/609 [==============================] - 184s 303ms/step - loss: 1.3038 - categorical_accuracy: 0.4153 - val_loss: 1.9181 - val_categorical_accuracy: 0.3097
Epoch 6/6
609/609 [==============================] - 184s 302ms/step - loss: 1.1962 - categorical_accuracy: 0.4612 - val_loss: 1.8710 - val_categorical_accuracy: 0.3

#### Fold4: intentionally separate cells for check point purpose.

In [33]:
from transformers import AutoTokenizer
import numpy as np

split0=splits[4]
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
SEQ_LEN=256
print(split0['X_train'].shape, split0['X_test'].shape)

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']

print("Xids_train.shape, Xids_test.shape: ",Xids_train.shape, Xids_test.shape)

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)
dataset_train = dataset_train.shuffle(42).batch(16)

train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

(10832,) (2708,)
Xids_train.shape, Xids_test.shape:  (10832, 256) (2708, 256)


In [34]:
from transformers import TFAutoModel



#balanced_accuracies_transfer_learning = []
#balanced_accuracies_fine_tuning = []
SEQ_LEN2=256


print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
, learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
#step1
bert_base = TFAutoModel.from_pretrained('bert-base-cased')
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = bert_base(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model1.layers[2].trainable = False
print(model1.summary())

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
#model2.summary() #Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_transfer_learning.append( balanced_acc )

#step5: fine tune
print("Fine tuning---------------")
model1.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_fine_tuning.append( balanced_acc )
print("----------------------------------------")
del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.3


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

609/609 [==============================] - 202s 306ms/step - loss: 1.7621 - categorical_accuracy: 0.3111 - val_loss: 2.1942 - val_categorical_accuracy: 0.2031
Epoch 2/6
609/609 [==============================] - 184s 302ms/step - loss: 1.6183 - categorical_accuracy: 0.3514 - val_loss: 2.0047 - val_categorical_accuracy: 0.2684
Epoch 3/6
609/609 [==============================] - 184s 302ms/step - loss: 1.5139 - categorical_accuracy: 0.3808 - val_loss: 1.9747 - val_categorical_accuracy: 0.2739
Epoch 4/6
609/609 [==============================] - 184s 302ms/step - loss: 1.4029 - categorical_accuracy: 0.4114 - val_loss: 1.9859 - val_categorical_accuracy: 0.2610
Epoch 5/6
609/609 [==============================] - 184s 302ms/step - loss: 1.3195 - categorical_accuracy: 0.4396 - val_loss: 1.9352 - val_categorical_accuracy: 0.2932
Epoch 6/6
609/609 [==============================] - 184s 302ms/step - loss: 1.2276 - categorical_accuracy: 0.4723 - val_loss: 1.9448 - val_categorical_accuracy: 0.3

### Take average

In [37]:
balanced_accuracies_fine_tuning

[0.33132053264155187, 0.3424170775321077]

In [ ]:
balanced_accuracy_fine_tune.append(0.3560173405159383)

In [ ]:
sum(balanced_accuracy_fine_tune)/5

### Conduct McNemar testing with the best parameter

In [ ]:
drop_out_rate = 0.2
learning_rate_transfer_learning = 1e-3
learning_rate_fine_tuning = 1e-5
McNemar = {}# (is_fine_tuning?, dropout_rate, learning_rate_transfer_learning, learning_rate_fine_tuning)

In [ ]:
from transformers import TFAutoModel

balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
SEQ_LEN2=256

drop_out_rate = 0.2
learning_rate_transfer_learning = 1e-3
learning_rate_fine_tuning = 1e-5


print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
, learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
#step1
roberta = TFAutoModel.from_pretrained('roberta-base')
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = roberta(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model2.layers[2].trainable = False
#model2.summary()

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
#model2.summary() #Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_transfer_learning.append( balanced_acc )

#step5: fine tune
print("Fine tuning---------------")
model2.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_fine_tuning.append( balanced_acc )
print("----------------------------------------")
del(model2)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.2


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 158s 234ms/step - loss: 2.1384 - categorical_accuracy: 0.2122 - val_loss: 2.8267 - val_categorical_accuracy: 0.0901
Epoch 2/6
609/609 [==============================] - 138s 227ms/step - loss: 1.9793 - categorical_accuracy: 0.2636 - val_loss: 2.3489 - val_categorical_accuracy: 0.1480
Epoch 3/6
609/609 [==============================] - 138s 227ms/step - loss: 1.9232 - categorical_accuracy: 0.2825 - val_loss: 2.4198 - val_categorical_accuracy: 0.1379
Epoch 4/6
609/609 [==============================] - 138s 227ms/step - loss: 1.8901 - categorical_accuracy: 0.2974 - val_loss: 2.3096 - val_categorical_accuracy: 0.1581
Epoch 5/6
609/609 [==============================] - 138s 227ms/step - loss: 1.8674 - categorical_accuracy: 0.2979 - val_loss: 2.2364 - val_categorical_accuracy: 0.1801
Epoch 6/6
609/609 [==============================] - 138s 227ms/step - loss: 1.8519 - categorical_accuracy: 0.3018 - val_loss: 2.2021 - val_categorical_acc

609/609 [==============================] - 357s 553ms/step - loss: 1.7825 - categorical_accuracy: 0.3348 - val_loss: 1.8293 - val_categorical_accuracy: 0.2960
Epoch 2/6
609/609 [==============================] - 333s 547ms/step - loss: 1.6608 - categorical_accuracy: 0.3509 - val_loss: 1.8948 - val_categorical_accuracy: 0.2923
Epoch 3/6
609/609 [==============================] - 333s 547ms/step - loss: 1.5689 - categorical_accuracy: 0.3829 - val_loss: 2.0517 - val_categorical_accuracy: 0.2491
Epoch 4/6
609/609 [==============================] - 333s 547ms/step - loss: 1.4769 - categorical_accuracy: 0.4039 - val_loss: 1.9109 - val_categorical_accuracy: 0.2978
Epoch 5/6
609/609 [==============================] - 333s 547ms/step - loss: 1.3818 - categorical_accuracy: 0.4348 - val_loss: 1.7944 - val_categorical_accuracy: 0.3529
Epoch 6/6
609/609 [==============================] - 333s 547ms/step - loss: 1.2942 - categorical_accuracy: 0.4637 - val_loss: 1.7972 - val_categorical_accuracy: 0.3

In [ ]:
McNemar_RoBERTa_split0_transfer_learning = pd.DataFrame(data= McNemar[(False,0.2, 1e-3,1e-5)] )
McNemar_RoBERTa_split0_transfer_learning.to_csv(DIR + 'McNemar_RoBERTa_split0_transfer_learning.csv')

McNemar_RoBERTa_split0_fine_tuning = pd.DataFrame(data= McNemar[(True, 0.2, 1e-3,1e-5)] )
McNemar_RoBERTa_split0_fine_tuning.to_csv(DIR + 'McNemar_RoBERTa_split0_fine_tuning.csv')

### Miscelaneous what if more than 512 tokens

In [ ]:
from transformers import AutoTokenizer
import numpy as np

split0=splits[0]
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_test'] = prepare_lyrics(split0['X_test'] )
#SEQ_LEN=256
SEQ_LEN=768
print(split0['X_train'].shape, split0['X_test'].shape)

tokenizer = AutoTokenizer.from_pretrained('roberta-base')

Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']

print("Xids_train.shape, Xids_test.shape: ",Xids_train.shape, Xids_test.shape)

labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)
dataset_train = dataset_train.shuffle(42).batch(16)

train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))
test = dataset_test.batch(16)

(10832,) (2708,)
Xids_train.shape, Xids_test.shape:  (10832, 768) (2708, 768)


In [ ]:
drop_out_rate = 0.3
learning_rate_transfer_learning =1e-3
learning_rate_fine_tuning = 1e-5

In [ ]:
McNemar.keys()

dict_keys([(False, 0.1, 0.001, 1e-05), (True, 0.1, 0.001, 1e-05), (False, 0.1, 0.001, 1e-06), (True, 0.1, 0.001, 1e-06), (False, 0.1, 0.0001, 1e-05), (True, 0.1, 0.0001, 1e-05), (False, 0.1, 0.0001, 1e-06), (True, 0.1, 0.0001, 1e-06), (False, 0.2, 0.001, 1e-05), (True, 0.2, 0.001, 1e-05), (False, 0.2, 0.001, 1e-06), (True, 0.2, 0.001, 1e-06), (False, 0.2, 0.0001, 1e-05), (True, 0.2, 0.0001, 1e-05), (False, 0.2, 0.0001, 1e-06), (True, 0.2, 0.0001, 1e-06), (False, 0.3, 0.001, 1e-05), (True, 0.3, 0.001, 1e-05), (False, 0.3, 0.001, 1e-06), (True, 0.3, 0.001, 1e-06), (False, 0.3, 0.0001, 1e-05), (True, 0.3, 0.0001, 1e-05), (False, 0.3, 0.0001, 1e-06), (True, 0.3, 0.0001, 1e-06), (False, 0.4, 0.001, 1e-05), (True, 0.4, 0.001, 1e-05), (False, 0.4, 0.001, 1e-06), (True, 0.4, 0.001, 1e-06), (False, 0.4, 0.0001, 1e-05), (True, 0.4, 0.0001, 1e-05), (False, 0.4, 0.0001, 1e-06), (True, 0.4, 0.0001, 1e-06), (False, 0.5, 0.001, 1e-05), (True, 0.5, 0.001, 1e-05), (False, 0.5, 0.001, 1e-06), (True, 0.5

In [ ]:
from transformers import TFAutoModel



#balanced_accuracies_transfer_learning = []
#balanced_accuracies_fine_tuning = []
SEQ_LEN2=SEQ_LEN


print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
, learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)
#step1
bert_base = TFAutoModel.from_pretrained('bert-base-cased')
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

embeddings = bert_base(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.BatchNormalization()(X)#added
X = tf.keras.layers.Dropout(drop_out_rate)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
X = tf.keras.layers.BatchNormalization()(X)#added
X = tf.keras.layers.Dropout(drop_out_rate)(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model1 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model1.layers[2].trainable = False
print(model1.summary())

#step2
optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)
#model2.summary() #Check trainable params increased.

#step3: transfer learning
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

#step4: predict
balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_transfer_learning.append( balanced_acc )

#step5: fine tune
print("Fine tuning---------------")
model1.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model1.compile(optimizer=optimizer, loss= loss, metrics=metrics)

history = model1.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
balanced_acc, McNemar=get_balanced_accuracy(model1, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
balanced_accuracies_fine_tuning.append( balanced_acc )
print("----------------------------------------")
del(model1)


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-05 drop_out_rate: 0.3


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 768)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 768)]        0           []                               
                                                                                                  
 tf_bert_model_4 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 768,                                         

ResourceExhaustedError: ignored